<a href="https://colab.research.google.com/github/prashantchandel97/Udacity_MLE_Notebooks/blob/main/Image_Captioning_Prashant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [31]:
import time
import tensorflow as tf
import pickle
import numpy as np
import os
import argparse
import sys
import json



Preprocess the image

In [59]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [60]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [65]:
image_features_extract_model.save('/gdrive/MyDrive/image_features_extract_model')

INFO:tensorflow:Assets written to: /gdrive/MyDrive/image_features_extract_model/assets


Preprocess and Tokenize the Captions

In [2]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [10]:
def load_doc(filename):
    # Opening the file as read only
    file = open(filename, 'r')

    # Reading all text and storing it.
    text = file.read()

    # Closing the file
    file.close()
    
    return text

In [22]:
def photo_to_description_mapping(descriptions):
    
    # Dictionary to store the mapping of photo identifiers to descriptions
    description_mapping = dict()
    
    # Iterating through each line of the descriptions
    for line in descriptions.split('\n'):
        
        # Splitting the lines by white space
        words = line.split()
        
        # Skipping the lines with length less than 2
        if len(line)<2:
            continue
            
        # The first word is the image_id and the rest are the part of the description of that image
        image_id, image_description = words[0], words[1:]
        
        # Retaining only the name of the image and removing the extension from it
        image_id = image_id.split('.')[0]
        
        # Image_descriptions contains comma separated words of the description, hence, converting it back to string
        image_description = ' '.join(image_description)
        image_description = '<start> '+image_description+' <end>'
        # There are multiple descriptions per image, 
         # hence, corresponding to every image identifier in the dictionary, there is a list of description
        # if the list does not exist then we need to create it
        
        if image_id not in description_mapping:
            description_mapping[image_id] = list()
            
        # Now storing the descriptions in the mapping
        description_mapping[image_id].append(image_description)
    
    return description_mapping

In [19]:
def clean_descriptions(description_mapping):
    
    # Preapring a translation table for removing all the punctuation
    table = str.maketrans('','', str.punctuation)
    
    # Traversing through the mapping we created
    for key, descriptions in description_mapping.items():
        for i in range(len(descriptions)):
            description = descriptions[i]
            description = description.split()
            
            # Converting all the words to lower case
            description = [word.lower() for word in description]
            
            # Removing the punctuation using the translation table we made
            description = [word.translate(table) for word in description]
            
            # Removing the words with length =1
            description = [word for word in description if len(word)>1]
            
            # Removing all words with number in them
            description = [word for word in description if word.isalpha()]
            
            # Converting the description back to string and overwriting in the descriptions list
            descriptions[i] = ' '.join(description)

In [23]:
filename = '/gdrive/MyDrive/train_data/train_data/captions.txt'
# Loading descriptions
doc = load_doc(filename)

# Parsing descriptions
descriptions = photo_to_description_mapping(doc)
print('Loaded: %d ' % len(descriptions))
# clean_descriptions(descriptions)

Loaded: 31784 


In [24]:
descriptions

{'image,caption': ['<start>  <end>'],
 '1000092795': ['<start> Two young guys with shaggy hair look at their hands while hanging out in the yard . <end>',
  '<start> Two young , White males are outside near many bushes ." <end>',
  '<start> Two men in green shirts are standing in a yard . <end>',
  '<start> A man in a blue shirt standing in a garden . <end>',
  '<start> Two friends enjoy time spent together . <end>'],
 '10002456': ['<start> Several men in hard hats are operating a giant pulley system . <end>',
  '<start> Workers look down from up above on a piece of equipment . <end>',
  '<start> Two men working on a machine wearing hard hats . <end>',
  '<start> Four men on top of a tall structure . <end>',
  '<start> Three men on a large rig . <end>'],
 '1000268201': ['<start> A child in a pink dress is climbing up a set of stairs in an entry way . <end>',
  '<start> A little girl in a pink dress going into a wooden cabin . <end>',
  '<start> A little girl climbing the stairs to her 

In [26]:
train_image_paths = list(descriptions.keys())

# Select the first 6000 image_paths from the shuffled set.
# Approximately each image id has 5 captions associated with it, so that will 
# lead to 30,000 examples.
# train_image_paths = image_paths
print(len(train_image_paths))

31784


In [27]:
train_captions = []
img_name_vector = []

for image_path in train_image_paths:
  caption_list = descriptions[image_path]
  train_captions.extend(caption_list)
  img_name_vector.extend([image_path] * len(caption_list))

In [32]:
# Choose the top 5000 words from the vocabulary
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)

In [33]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [34]:
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [35]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [36]:
# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)

In [44]:

class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

        # hidden shape == (batch_size, hidden_size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # attention_hidden_layer shape == (batch_size, 64, units)
        attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                             self.W2(hidden_with_time_axis)))

        # score shape == (batch_size, 64, 1)
        # This gives you an unnormalized score for each image feature.
        score = self.V(attention_hidden_layer)

        # attention_weights shape == (batch_size, 64, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x


class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        # defining attention as a separate model
        context_vector, attention_weights = self.attention(features, hidden)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # shape == (batch_size, max_length, hidden_size)
        x = self.fc1(output)

        # x shape == (batch_size * max_length, hidden_size)
        x = tf.reshape(x, (-1, x.shape[2]))

        # output shape == (batch_size * max_length, vocab)
        x = self.fc2(x)

        return x, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64
loss_plot = []


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# @tf.function
def train_step(img_tensor, target):
    loss = 0

    # initializing the hidden state for each batch
    # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([3] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)

            loss += loss_function(target[:, i], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

In [ ]:
3070130228.npy
[   3   13   17   19   27   31    8   22  182  352 2144   66   62 1671
    5  113    4    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]

In [5]:
def map_func(img_name, cap):
    path = str(img_name.decode('utf-8'))
#     path = path.split('.')
#     path = '.'+path[1]+'.npy'
#     path = path.split('/')[2]
    data_dir = '/gdrive/MyDrive/train_data/train_data'
    data_path = os.path.join(data_dir,'Images_numpy')
    new_path = os.path.join(data_path,path)
#     print(new_path)
    if(len(cap)!=80):
      print(len(cap))
    try:
      img_tensor = np.load(new_path)
      return img_tensor, cap
    except:
      img_tensor = np.load('/gdrive/MyDrive/train_data/train_data/Images_numpy/3070130228.npy')
      return img_tensor, cap_def
    return

In [10]:
with open('/gdrive/MyDrive/train_data/train_data/img_name_train_file', 'rb') as fp:
    img_name_train = pickle.load(fp)[15000:25000]
with open('/gdrive/MyDrive/train_data/train_data/cap_train_file', 'rb') as fp:
    cap_train = pickle.load(fp)[15000:25000]



In [6]:
# Run only once
cap_def = cap_train[0]

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))
print(img_name_train[0])
print(cap_train[0])
# Use map to load the numpy files in parallel

dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]))
# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=BUFFER_SIZE)

#     checkpoint_path = args.model_dir
vocab_size = 5000
num_steps = len(img_name_train)

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

checkpoint_path = '/gdrive/MyDrive/'
ckpt = tf.train.Checkpoint(encoder=encoder,
                            decoder=decoder,
                            optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

#     train(start_epoch, dataset, ckpt_manager, num_steps,'/opt/ml/model')
#     train(start_epoch, dataset, num_steps)

EPOCHS = 25

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0
    # ckpt_manager.save()
    for (batch, (img_tensor, target)) in enumerate(dataset):
        # print(img_tensor)
        
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 50 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    ckpt_manager.save()

print('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                    total_loss / num_steps))
print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

2517637587.npy
[   3    2   10    5    2  505   23    1   20 1087   28 1960   16    2
  186   11  466    7    2  301  112    4    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]
Epoch 16 Batch 0 Loss 0.6868
Epoch 16 Batch 50 Loss 0.5352
Epoch 16 Batch 100 Loss 0.5572
Epoch 16 Batch 150 Loss 0.5824
Epoch 17 Batch 0 Loss 0.5725
Epoch 17 Batch 50 Loss 0.3966
Epoch 17 Batch 100 Loss 0.4948
Epoch 17 Batch 150 Loss 0.4395
Epoch 18 Batch 0 Loss 0.3985
Epoch 18 Batch 50 Loss 0.4596
Epoch 18 Batch 100 Loss 0.5091
Epoch 18 Batch 150 Loss 0.4155
Epoch 19 Batch 0 Loss 0.4075
Epoch 19 Batch 50 Loss 0.3813
Epoch 19 Batch 100 Loss 0.3918
Epoch 19 Batch 150 Loss 0.3528
Epoch 20 Batch 0 Loss 0.3859
Epoch 20 Batch 50 Loss 0.2957
Epoch 20 Batch 100 Loss 0.3451
Epoc

In [51]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

87916544/87910968 [==============================] - 1s 0us/step


In [38]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [56]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, 5000)
optimizer = tf.keras.optimizers.Adam()
checkpoint_path = '/gdrive/MyDrive/'
ckpt = tf.train.Checkpoint(encoder=encoder,
                            decoder=decoder,
                            optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

In [49]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [57]:
# rid = np.random.randint(0, len(img_name_val))
# image = img_name_val[rid]
# real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate('/content/1000366164.jpg')

# print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))

Prediction Caption: a is lab a hikers kiosk <unk> a pedestrian man crowded at a clear white crowded bench and match shoes large practice mother girl tide <end>
